In [ ]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
# Đường dẫn đến tập tin CSV
file_path = 'C:\\Users\\Dell\\OneDrive\\Desktop\\BD_DQ\\data_dotquy.csv'
# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv(file_path)
# Hiển thị DataFrame trong VS Code
print(df)

In [ ]:
#Loại bỏ giới tính mang giá trị Other
mask = df['gender'].isin(['Male', 'Female'])
df = df[mask]
print(df)

In [ ]:
# gender: female 1 male 0
df.loc[:, 'gender'] = df['gender'].replace({'Female': 1, 'Male': 0})
print(df)

In [ ]:
# Residence_type: urban 1 rural 0
df.loc[:, 'Residence_type'] = df['Residence_type'].replace({'Urban': 1, 'Rural': 0})
print(df)

In [ ]:
import numpy as np
#chuẩn hóa dữ liệu field work_type theo kỹ thuật label encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df.loc[:, 'work_type'] = le.fit_transform(df['work_type'])
print(df)